In [ ]:
import torch
from diffusers import AnimateDiffPipeline, DDIMScheduler, MotionAdapter
from diffusers.utils import export_to_gif
import imageio

In [ ]:
adapter = MotionAdapter.from_pretrained(
    "guoyww/animatediff-motion-adapter-v1-5-2",
    torch_dtype=torch.float16
)

In [ ]:
model_id = "SG161222/Realistic_Vision_V5.1_noVAE"
pipe = AnimateDiffPipeline.from_pretrained(
    model_id,
    motion_adapter=adapter,
    torch_dtype=torch.float16
)

In [ ]:
scheduler = DDIMScheduler.from_pretrained(
    model_id,
    subfolder="scheduler",
    clip_sample=False,
    timestep_spacing="linspace",
    beta_schedule="linear",
    steps_offset=1,
)

pipe.scheduler = scheduler
pipe.enable_vae_slicing()
pipe.enable_model_cpu_offload()

In [ ]:
prompt = (
     "A vibrant Indian wedding procession with groom on horse, colorful traditional sarees and sherwanis, festive decorations, street dancing, cinematic lighting, 3D animation style"
)
negative_prompt = "blurry, distorted, low quality, deformed, bad anatomy, low resolution"

output = pipe(
    prompt=prompt,
    negative_prompt=negative_prompt,
    num_frames=32,
    guidance_scale=7.5,
    num_inference_steps=27,
    generator=torch.Generator("cuda").manual_seed(42)
)

In [ ]:
# Export GIF
frames = output.frames[0]
export_to_gif(frames, "Indian Wedding.gif")
print("GIF saved: Indian Wedding.gif")


In [ ]:
# Export MP4
imageio.mimsave("Indian Wedding.mp4", frames, fps=6)
print("MP4 saved: Indian Wedding.mp4")


In [ ]:
frames = 32
fps = 6
duration = frames / fps
print(f"Duration: {duration:.2f} seconds")
